<a href="https://colab.research.google.com/github/pavannanaiah/movies/blob/main/Moveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Collection


In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from numpy import NAN



In [98]:
# Basic Data Collection

def imdb_bacic_movie_data(start_fetch,number_of_records_per_search,start_last_page,number_of_votes):

  # start_fetch = 1
  # number_of_records_per_search = 50
  # start_last_page = 201
  # number_of_votes = 70

  movie_data = pd.DataFrame()
  movie_data_subset = pd.DataFrame()

  while start_fetch <= start_last_page:

    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2021-01-01,2023-09-30&num_votes="+str(number_of_votes)+",&count="+str(number_of_records_per_search)+"&start=" +str(start_fetch)+"&ref_=adv_nxt"
    #print (url)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        movie_headers = soup.find_all('div', class_='lister-item mode-advanced')

        movie_ids = []
        movie_names = []
        movie_genres = []
        movie_years = []
        movie_ratings = []
        movie_durations = []
        movie_votes = []

        for header in movie_headers:

            href = header.find('a')['href']
            movie_id = href.split('/')[2] if href.startswith('/title/') else ''

            movie_name = header.find('h3', class_='lister-item-header').find('a').text.strip()
            movie_genre = header.find('p', class_='text-muted').find('span', class_='genre').text.strip()
            movie_year = header.find('span', class_='lister-item-year').text.strip('()')

            rating_text = header.find('div', class_="inline-block ratings-imdb-rating", attrs={"name":"ir"})
            movie_rating = ''#rating_text['data-value'] if vote_text else ''

            movie_duration = header.find('p', class_='text-muted').find('span', class_='runtime').text.strip()

            vote_text = header.find('span', attrs={"name":"nv"})
            movie_vote = vote_text['data-value'] if vote_text else ''

            movie_ids.append(movie_id)
            movie_names.append(movie_name)
            movie_genres.append(movie_genre)
            movie_years.append(movie_year)
            movie_ratings.append(movie_rating)
            movie_durations.append(movie_duration)
            movie_votes.append(movie_vote)


        movie_data_subset = pd.DataFrame({'Movie ID':movie_ids,'Movie Name': movie_names,
                                  'Movie Genre': movie_genres, 'Movie Year': movie_years,
                                  'Movie Rating':movie_ratings,'Movie Duration': movie_durations,
                                  'Movie Votes': movie_votes})
        movie_data = pd.concat([movie_data, movie_data_subset], ignore_index=True)
        #print(movie_data)
    else:
      movie_data = None

    start_fetch += number_of_records_per_search

  return movie_data

In [73]:

# Collect Budget , Revenue Data and Language
def imdb_extended_movie_data(movie_data):
  movie_languages = []
  movie_revenues = []
  movie_budgets = []


  for index, movie in movie_data.iterrows():
    url = "https://www.imdb.com/title/" + movie["Movie ID"]
    #print(url)

    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

    response = requests.get(url, headers= HEADERS)

    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')

      money_element = soup.find("div", {"data-testid": "title-boxoffice-section"})
      languages = soup.find("li", {"data-testid": "title-details-languages"})
      #print (languages)
      if (money_element != None):
        #For budget
        if money_element.find("li", {"data-testid": "title-boxoffice-budget"}) != None:
          budget = money_element.find("li", {"data-testid": "title-boxoffice-budget"}).text.strip()
        else:
          budget = " "
        #print(budget)

      #For revenue
        if money_element.find("li", {"data-testid": "title-boxoffice-cumulativeworldwidegross"}) != None:
          revenue =  money_element.find("li", {"data-testid": "title-boxoffice-cumulativeworldwidegross"}).text.strip()
        else:
          revenue = " "

        #For langauges
        language_links = languages.find_all("a", {"class": "ipc-metadata-list-item__list-content-item--link"})
        languages = ', '.join([link.text.strip() for link in language_links])

        movie_budget = budget
        movie_revenue = revenue
        movie_language = languages
      else:
        movie_budget = " "
        movie_revenue = " "

      movie_budgets.append(movie_budget.replace("Budget","").replace(" (estimated)","").replace("$",""))
      movie_revenues.append(movie_revenue.replace("Gross worldwide","").replace("$",""))
      movie_languages.append(languages)

    else:
      movie_budgets.append(" ")
      movie_revenues.append(" ")
      movie_languages.append(" ")


  movie_data['Movie Budget'] = movie_budgets
  movie_data['Movie Revenue'] = movie_revenues
  movie_data['Movie Languages'] = movie_languages

  return movie_data



In [97]:
movie_data = imdb_bacic_movie_data(1,50,1,70)

movie_data_extended = imdb_extended_movie_data(movie_data)

movie_data_extended.head()

https://www.imdb.com/search/title/?title_type=feature&release_date=2021-01-01,2023-09-30&num_votes=70,&count=50&start=1&ref_=adv_nxt


,Movie ID,Movie Name,Movie Genre,Movie Year,Movie Rating,Movie Duration,Movie Votes,Movie Budget,Movie Revenue,Movie Languages
0,tt11858890,The Creator,"Action, Adventure, Drama",2023,,133 min,23906,"80,000,000","36,355,158",English
1,tt14509110,No One Will Save You,"Horror, Sci-Fi, Thriller",2023,,93 min,28672,,,"[[Language], [[<li class=""ipc-inline-list__ite..."
2,tt21807222,Saw X,"Horror, Mystery, Thriller",2023,,118 min,10726,"13,000,000","34,574,158",English
3,tt3291150,Expend4bles,"Action, Adventure, Thriller",2023,,103 min,8283,"100,000,000","24,549,662",English
4,tt1517268,Barbie,"Adventure, Comedy, Fantasy",I) (2023,,114 min,346318,"100,000,000","1,434,328,670","English, Spanish"
